In [8]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures, LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split

from prettytable import PrettyTable

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [9]:
aug_data = pd.read_json('../data/aug.json', convert_dates=True)
sep_data = pd.read_json('../data/sept.json', convert_dates=True)
oct_data = pd.read_json('../data/oct.json', convert_dates=True)
nov_data = pd.read_json('../data/nov.json', convert_dates=True)

rec_data = pd.read_json('../data/mod_recovery_data.json', convert_dates=True)

In [146]:
aug_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   facility      123 non-null    object        
 1   date          123 non-null    datetime64[ns]
 2   timeStart     123 non-null    object        
 3   timeEnd       123 non-null    object        
 4   supplierCode  123 non-null    object        
 5   suppliedM3    123 non-null    float64       
 6   recoveredM3   123 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 6.9+ KB


In [147]:

for feat in sep_data.columns.to_list():
    if feat not in aug_data.columns.to_list():
        print(f'{feat} missing from aug_data dataset')
print('\n')

for feat in oct_data.columns.to_list():
    if feat not in aug_data.columns.to_list():
        print(f'{feat} missing from aug_data dataset')
print('\n')

for feat in nov_data.columns.to_list():
    if feat not in aug_data.columns.to_list():
        print(f'{feat} missing from aug_data dataset')

print('\n')
for feat in sep_data.columns.to_list():
    if feat not in rec_data.columns.to_list():
        print(f'{feat} missing from aug_data dataset')

processTime missing from aug_data dataset
supplier missing from aug_data dataset


processTime missing from aug_data dataset
supplier missing from aug_data dataset


processTime missing from aug_data dataset
supplier missing from aug_data dataset


timeStart missing from aug_data dataset
processTime missing from aug_data dataset
suppliedM3 missing from aug_data dataset
recoveredM3 missing from aug_data dataset
timeEnd missing from aug_data dataset
supplierCode missing from aug_data dataset


suppliedM3, recoveredM3 are missing from rec_data where it is present volumeM3, i assume that volume consist in the volume of recoveredM3 hence is the same.. probably will be useful to investigate distributions.

timeStart and timeEnd are missing from rec_data, it is present a time feature, however i dont know to what is related.

is missing also the processTime and suplierCode, since the supplierCode is possible to recover from before datasets process time i have no idea how to find it at this moment.

In [148]:
aug_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   facility      123 non-null    object        
 1   date          123 non-null    datetime64[ns]
 2   timeStart     123 non-null    object        
 3   timeEnd       123 non-null    object        
 4   supplierCode  123 non-null    object        
 5   suppliedM3    123 non-null    float64       
 6   recoveredM3   123 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 6.9+ KB


In [149]:
sep_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   facility      232 non-null    object        
 1   timeStart     232 non-null    object        
 2   processTime   111 non-null    object        
 3   supplier      111 non-null    object        
 4   suppliedM3    232 non-null    float64       
 5   recoveredM3   232 non-null    float64       
 6   date          121 non-null    datetime64[ns]
 7   timeEnd       121 non-null    object        
 8   supplierCode  121 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 16.4+ KB


In [150]:
sep_data.nunique()

facility          2
timeStart       220
processTime      38
supplier          4
suppliedM3      120
recoveredM3     129
date             29
timeEnd         107
supplierCode      3
dtype: int64

In [151]:
all_ds = pd.concat([aug_data, sep_data, oct_data, nov_data])

In [152]:
ptds = all_ds[~all_ds.processTime.isna()]
ptds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 0 to 226
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   facility      329 non-null    object        
 1   date          0 non-null      datetime64[ns]
 2   timeStart     329 non-null    object        
 3   timeEnd       0 non-null      object        
 4   supplierCode  0 non-null      object        
 5   suppliedM3    329 non-null    float64       
 6   recoveredM3   329 non-null    float64       
 7   processTime   329 non-null    object        
 8   supplier      329 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 25.7+ KB


In [153]:
ptds = all_ds[~all_ds.timeEnd.isna()]
ptds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 481 entries, 0 to 227
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   facility      481 non-null    object        
 1   date          481 non-null    datetime64[ns]
 2   timeStart     481 non-null    object        
 3   timeEnd       481 non-null    object        
 4   supplierCode  481 non-null    object        
 5   suppliedM3    481 non-null    float64       
 6   recoveredM3   481 non-null    float64       
 7   processTime   0 non-null      object        
 8   supplier      0 non-null      object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 37.6+ KB


In [154]:
# pd.to_datetime(ptds.timeEnd, format='%I:%M:%S %p')
def convert_to_hours(delta):
    total_seconds = delta.total_seconds()
    hours = str(int(total_seconds // 3600)).zfill(2)
    minutes = str(int((total_seconds % 3600) // 60)).zfill(2)
    return f"{hours}:{minutes}"

all_ds['processTime'] = all_ds.apply(lambda x: convert_to_hours((pd.to_datetime(x.timeEnd, format='%I:%M:%S %p') - pd.to_datetime(x.timeStart, format='%I:%M:%S %p'))) if pd.isna(x.processTime) else x.processTime , axis=1)

In [155]:
all_ds.supplier.unique()

array([nan, 'Mary', 'Mary Therese', 'Mary Jane', 'Mary Anne'],
      dtype=object)

In [156]:
all_ds['supplier_id'] = all_ds.apply(lambda x: x.supplierCode if pd.isna(x.supplier) else x.supplier, axis=1)

In [157]:
all_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810 entries, 0 to 227
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   facility      810 non-null    object        
 1   date          481 non-null    datetime64[ns]
 2   timeStart     810 non-null    object        
 3   timeEnd       481 non-null    object        
 4   supplierCode  481 non-null    object        
 5   suppliedM3    810 non-null    float64       
 6   recoveredM3   810 non-null    float64       
 7   processTime   810 non-null    object        
 8   supplier      329 non-null    object        
 9   supplier_id   810 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 69.6+ KB


not all dates are filled, however it is possibile to  see that those that are absent have the full date in the timeStart.
hence i can easily fill the gaps

In [158]:
all_ds['date'] = all_ds.apply(lambda x: pd.to_datetime(x.timeStart).date() if pd.isna(x.date) else pd.to_datetime(x.date).date() , axis=1)

In [159]:
all_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810 entries, 0 to 227
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   facility      810 non-null    object 
 1   date          810 non-null    object 
 2   timeStart     810 non-null    object 
 3   timeEnd       481 non-null    object 
 4   supplierCode  481 non-null    object 
 5   suppliedM3    810 non-null    float64
 6   recoveredM3   810 non-null    float64
 7   processTime   810 non-null    object 
 8   supplier      329 non-null    object 
 9   supplier_id   810 non-null    object 
dtypes: float64(2), object(8)
memory usage: 69.6+ KB


In [160]:
all_ds.drop(columns=[ 'supplierCode', 'supplier'], inplace=True)

In [161]:
all_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810 entries, 0 to 227
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   facility     810 non-null    object 
 1   date         810 non-null    object 
 2   timeStart    810 non-null    object 
 3   timeEnd      481 non-null    object 
 4   suppliedM3   810 non-null    float64
 5   recoveredM3  810 non-null    float64
 6   processTime  810 non-null    object 
 7   supplier_id  810 non-null    object 
dtypes: float64(2), object(6)
memory usage: 57.0+ KB


In [162]:
all_ds.nunique()

facility         2
date           120
timeStart      614
timeEnd        325
suppliedM3     199
recoveredM3    213
processTime    233
supplier_id      7
dtype: int64

I'll make some training and predictions on the past data with both one hot hencoding and label encoding in order to see if the features will have make tangible differences.

In [163]:
ds_dummies = pd.get_dummies(all_ds, columns=['facility', 'supplier_id'])

ds_dummies.drop(columns=['timeStart', 'timeEnd'], inplace=True)

In [164]:
ds_dummies['processTime'] = ds_dummies[ 'processTime'] + ':00'

In [165]:
ds_dummies['processTime'] = ds_dummies.apply(lambda x: pd.Timedelta(x.processTime)/pd.Timedelta(minutes=60), axis=1)

In [166]:
ds_dummies.drop(columns=['date'], inplace=True)

In [167]:
x = ds_dummies.drop(columns=['recoveredM3'])
y = ds_dummies['recoveredM3']


In [168]:
degree = 2

In [169]:
models = [LinearRegression(),
          Ridge(random_state=42),
          MLPRegressor(hidden_layer_sizes=(10,), random_state=42, max_iter=10000),
          SVR(gamma='scale'),
          RandomForestRegressor(n_estimators=300)
          ]

names = [
        'linreg',
        'ridge',
        'mlp',
        'svr',
        'rf'
    ]

In [170]:
t = PrettyTable()
t.field_names = ['model', 'MSE', 'R2']

for model, name in zip(models, names):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.8, random_state=42, shuffle=True)

    model.fit(x_train, y_train)
    y_hat = model.predict(x_test)

    mse = mean_squared_error(y_test, y_hat)
    r2 = r2_score(y_test, y_hat)
    t.add_row([name, mse, r2])

print(t)

+------------------+---------------------+--------------------+
|      model       |         MSE         |         R2         |
+------------------+---------------------+--------------------+
|      linreg      | 0.11170385253720046 | 0.8720620164662826 |
|      ridge       | 0.11010811602252846 | 0.8738896643701027 |
|       mlp        | 0.17177785449937077 | 0.8032573468038724 |
|       svr        | 0.11516316516789216 | 0.8680999554242426 |
|        rf        | 0.13387013485128277 | 0.8466742666502378 |
| sin+poly2+linreg | 0.12056584623078706 | 0.8619120925604047 |
| sin+poly2+ridge  | 0.12137536851207002 | 0.8609849208833269 |
+------------------+---------------------+--------------------+


with the data i have the models are working kinda well

In [171]:
all_ds.supplier_id.unique(), all_ds.facility.unique()

(array(['har', 'dic', 'tom', 'Mary', 'Mary Therese', 'Mary Jane',
        'Mary Anne'], dtype=object),
 array(['Newcastle', 'Bundaberg'], dtype=object))

In [172]:
labelencoder = LabelEncoder()

all_ds.supplier_id = labelencoder.fit_transform(all_ds.supplier_id)


In [173]:
all_ds.facility = labelencoder.fit_transform(all_ds.facility)

In [174]:
label_encoded_ds = all_ds[['facility', 'suppliedM3', 'recoveredM3', 'processTime', 'supplier_id']]

label_encoded_ds['processTime'] = label_encoded_ds[ 'processTime'] + ':00'
label_encoded_ds['processTime'] = label_encoded_ds.apply(lambda x: pd.Timedelta(x.processTime)/pd.Timedelta(minutes=60), axis=1)

<ipython-input-174-77197d6d7828>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_encoded_ds['processTime'] = label_encoded_ds[ 'processTime'] + ':00'
<ipython-input-174-77197d6d7828>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_encoded_ds['processTime'] = label_encoded_ds.apply(lambda x: pd.Timedelta(x.processTime)/pd.Timedelta(minutes=60), axis=1)


In [175]:
# creating some base models in order to understand how those data would be usefull to predict and if prediction might be good

x = label_encoded_ds.drop(columns=['recoveredM3'])
y = label_encoded_ds['recoveredM3']


In [176]:
t = PrettyTable()
t.field_names = ['model', 'MSE', 'R2']


for model, name in zip(models, names):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.8, random_state=42, shuffle=True)

    model.fit(x_train, y_train)
    y_hat = model.predict(x_test)

    mse = mean_squared_error(y_test, y_hat)
    r2 = r2_score(y_test, y_hat)
    t.add_row([name, mse, r2])

print(t)

+------------------+---------------------+---------------------+
|      model       |         MSE         |          R2         |
+------------------+---------------------+---------------------+
|      linreg      | 0.11320081065711592 |  0.8703475026071242 |
|      ridge       |  0.1127020910309505 |  0.8709187020946176 |
|       mlp        | 0.13584958419421003 |  0.8444071402110945 |
|       svr        | 0.11563866614389448 |  0.8675553489970131 |
|        rf        | 0.13517859755375033 |  0.8451756426020921 |
| sin+poly2+linreg |  0.8486608522420392 | 0.02800166982876562 |
| sin+poly2+ridge  |  0.8489494092324974 | 0.02767117630812832 |
+------------------+---------------------+---------------------+


In [177]:
all_ds.drop(columns=['date'], inplace=True)


In [178]:
all_ds['processTime'] = ds_dummies['processTime']

In [179]:
all_ds['mean'] = all_ds.groupby(by=['facility', 'supplier_id'])['processTime'].transform('mean')
all_ds['mean_by_facility'] = all_ds.groupby(by=['facility'])['processTime'].transform('mean')
all_ds['mean_by_supplier'] = all_ds.groupby(by=['supplier_id'])['processTime'].transform('mean')

after loking at the recovery data to predict it is possible to see that there are no endTime feature hence it is not possible to have the information of the processTime, we can estimate an average processTime by facility and supplier, add these info to the data and see how the model works.

In [180]:
# all_ds['timeStart'] = all_ds.apply(lambda x: pd.Timedelta(x.timeStart)/pd.Timedelta(minutes=60), axis=1)
ds_avg = all_ds[['facility', 'suppliedM3', 'supplier_id', 'recoveredM3', 'mean_by_supplier', 'mean_by_facility']]

x = ds_avg.drop(columns=['recoveredM3'])
y = ds_avg['recoveredM3']

In [181]:
t = PrettyTable()
t.field_names = ['model', 'MSE', 'R2']


for model, name in zip(models, names):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.6, random_state=42, shuffle=True)

    model.fit(x_train, y_train)
    y_hat = model.predict(x_test)

    mse = mean_squared_error(y_test, y_hat)
    r2 = r2_score(y_test, y_hat)
    t.add_row([name, mse, r2])

print(t)

+------------------+---------------------+----------------------+
|      model       |         MSE         |          R2          |
+------------------+---------------------+----------------------+
|      linreg      | 0.11628988199371496 |  0.8696836699837165  |
|      ridge       | 0.11403074139625034 |  0.8722152996199758  |
|       mlp        | 0.10756495408571742 |  0.879460965868223   |
|       svr        | 0.11292757295911378 |  0.8734515280833017  |
|        rf        | 0.14370566866792048 |  0.8389610942734335  |
| sin+poly2+linreg |   0.86451764269782  | 0.031207492008656024 |
| sin+poly2+ridge  |  0.8645839054018672 | 0.031133236946561782 |
+------------------+---------------------+----------------------+


with the new feature created we can see that the MLPregressor works best, hence i decide to use that model to predict the recovered volumes.

Now i'll prepare the data for the prediction, in order to have same structure and data with the trained model

In [182]:
rec_data['facility'] = labelencoder.fit_transform(rec_data.facility)

In [183]:
rec_data.drop(columns=['date', 'time'], inplace=True)

In [184]:
# all_ds[['facility', 'supplier_id', 'mean']]

,facility,supplier_id,mean
0,1,5,0.887957
1,1,4,2.654192
2,1,5,0.887957
3,1,6,1.836708
4,1,4,2.654192
...,...,...,...
223,0,1,1.318182
224,0,0,2.425556
225,1,6,1.836708
226,0,3,1.830616


In [185]:
rc = rec_data.copy()
rc['supplier'] = labelencoder.fit_transform(rc.supplier)

with the average process time found before by facility and suplier i add them to the data that has to be used for the prediction.

In [186]:
tomgds = all_ds[['facility', 'mean_by_facility']]

tomgds.columns = ['facility', 'mean_by_facility']
tomgds.drop_duplicates(inplace=True)
rec = pd.merge(rc, tomgds, how='left',  on=['facility'])

tomgds = all_ds[['supplier_id', 'mean_by_supplier']]

tomgds.columns = ['supplier', 'mean_by_supplier']
tomgds.drop_duplicates(inplace=True)
rec = pd.merge(rec, tomgds, how='left',  on=['supplier'])


<ipython-input-186-53742cd76f50>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tomgds.drop_duplicates(inplace=True)
<ipython-input-186-53742cd76f50>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tomgds.drop_duplicates(inplace=True)


In [195]:
ml = MLPRegressor(hidden_layer_sizes=(90,), random_state=53, max_iter=10000)

ml.fit(x, y)

rec_data['predicted'] = ml.predict(rec)

In [197]:
rec_data.to_csv('predicted_recovered')